In [1]:
import pandas as pd

In [2]:
# how often does a 20 beat a dealer with a 5 when dealer hits on soft 17
import blackjack_utils.game_config as gc
import blackjack_utils.shoe as shoe
import blackjack_utils.card as card
import random

# create a shoe with 6 decks
deck = shoe.Shoe(6)
player_cards = [card.Card().from_ints(11, 0), card.Card().from_ints(9, 0)]
dealer_card_up = card.Card().from_ints(3, 0)
deck.remove(player_cards[0])
deck.remove(player_cards[1])
deck.remove(dealer_card_up)
game_config = gc.GameConfig(6, True, True, True, 1.5)
results = []
for i in range(100000):
    deck_copy = shoe.Shoe(6)
    deck_copy.cards = deck.cards.copy()
    deck_copy.shuffle()
    dealer_cards = [dealer_card_up, deck_copy.draw()]
    result = game_config.evaluate(player_cards, dealer_cards, deck_copy)
    results.append(result)


In [3]:
mean = sum(results) / len(results)
print(mean) # 0.67191 - interpretation: expected value is + 0.67 - you profit 67% of your bet on average

0.66765


In [4]:
# now what happens when the player stays on 17 vs an 8
deck = shoe.Shoe(6)
player_cards = [card.Card().from_ints(11, 0), card.Card().from_ints(5, 0)]
dealer_card_up = card.Card().from_ints(6, 0)
deck.remove(player_cards[0])
deck.remove(player_cards[1])
deck.remove(dealer_card_up)
game_config = gc.GameConfig(6, True, True, True, 1.5)
results = []
for i in range(100000):
    deck_copy = shoe.Shoe(6)
    deck_copy.cards = deck.cards.copy()
    deck_copy.shuffle()
    dealer_cards = [dealer_card_up, deck_copy.draw()]
    result = game_config.evaluate(player_cards, dealer_cards, deck_copy)
    results.append(result)


In [5]:
mean = sum(results) / len(results)
print(mean) # -0.38408 - interpretation: expected value is -0.38408 - you lose 38% of your bet on average

-0.38675


In [ ]:
# gives a breakdown of the EV when a player stands on 17 vs different dealer cards
data = pd.DataFrame(columns=['player_total', 'dealer_card_up', 'expected_value'])
player_cards = [card.Card().from_ints(11, 0), card.Card().from_ints(5, 0)]

game_config = gc.GameConfig(6, True, True, True, 1.5)
for i in list(range(9)) + [12]:
    dealer_card_up = card.Card().from_ints(i, 0)
    deck = shoe.Shoe(6)
    deck.remove(player_cards[0])
    deck.remove(player_cards[1])
    deck.remove(dealer_card_up)
    results = []
    for i in range(40000):
        deck_copy = shoe.Shoe(6)
        deck_copy.cards = deck.cards.copy()
        deck_copy.shuffle()
        dealer_cards = [dealer_card_up, deck_copy.draw()]
        result = game_config.evaluate(player_cards, dealer_cards, deck_copy)
        results.append(result)

    mean = sum(results) / len(results)
    data = pd.concat([data, pd.DataFrame({'player_total': [game_config.score_hand(player_cards)], 'dealer_card_up': [dealer_card_up.get_card_value()], 'expected_value': [mean]})], ignore_index=True)
data

C:\Users\amarl\AppData\Local\Temp\ipykernel_23372\896386972.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data = pd.concat([data, pd.DataFrame({'player_total': [game_config.score_hand(player_cards)], 'dealer_card_up': [dealer_card_up.get_card_value()], 'expected_value': [mean]})], ignore_index=True)


,player_total,dealer_card_up,expected_value
0,17,2,-0.157150
1,17,3,-0.114375
2,17,4,-0.086575
3,17,5,-0.045950
4,17,6,-0.011725
5,17,7,-0.107300
6,17,8,-0.387650
7,17,9,-0.419225
8,17,10,-0.456475
9,17,11,-0.662075


In [ ]:
# next up - use logic similar to the above to determine what the player should do in any given situation
# start with player has 20 vs each possible dealer card up, analyze the options (hit, stay, double down)
# with lower player totals, if a player hits, use the prescribed action after the hit for the new higher total
# will have to do a second pass to figure out when to split and what to do with soft totals